<a href="https://colab.research.google.com/github/auromaa-2004/Logistic_Reg/blob/main/Credit_Card_Fraud_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
df=pd.read_csv("/content/PS_20174392719_1491204439457_log.csv")

In [ ]:
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0.0,0.0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0.0,0.0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1.0,0.0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1.0,0.0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0.0,0.0


In [ ]:
df.isnull().sum()

step              0
type              0
amount            1
nameOrig          1
oldbalanceOrg     1
newbalanceOrig    1
nameDest          1
oldbalanceDest    1
newbalanceDest    1
isFraud           1
isFlaggedFraud    1
dtype: int64

In [ ]:
df=df.dropna().reset_index(drop=True)

In [ ]:
df.isnull().sum()

step              0
type              0
amount            0
nameOrig          0
oldbalanceOrg     0
newbalanceOrig    0
nameDest          0
oldbalanceDest    0
newbalanceDest    0
isFraud           0
isFlaggedFraud    0
dtype: int64

In [ ]:
df["isFraud"].value_counts()

isFraud
0.0    56102
1.0      100
Name: count, dtype: int64

In [ ]:
df.shape

(56202, 11)

In [ ]:
#segregation into dependent and independent featured
X=df[['amount', 'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest']]
Y=df["isFraud"]

In [ ]:
#dividing into training and testing data
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.1,random_state=5)

In [ ]:
#standardising numeric data
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
x_train_scaled=scaler.fit_transform(x_train)
x_test_scaled=scaler.transform(x_test)

LOGISTIC REGRESSION

In [ ]:
#training logistic model
from sklearn.linear_model import LogisticRegression
logi=LogisticRegression()
logi.fit(x_train_scaled,y_train)

LogisticRegression()

In [ ]:
y_pred=logi.predict(x_test_scaled)

In [ ]:
#checking performance matrix of the model
from sklearn.metrics import confusion_matrix, accuracy_score
cm=confusion_matrix(y_test,y_pred)
score=accuracy_score(y_test,y_pred)
print(cm)
print(score)

[[5610    0]
 [   9    2]]
0.9983988614125601


In [ ]:
#hyperparameter tuning and cross validation
model=LogisticRegression()
penalty=["l1","l2","ElasticNet"]
c=[100,10,1.0,0.1,0.01]
class_weight=[{0:w,1:y} for w in [1,10,50,100,500,1000] for y in [1,10,50,100,500,1000]]
param=dict(
    penalty=penalty,
    C=c,
    class_weight=class_weight
)

#stratified k fold cv
from sklearn.model_selection import StratifiedKFold
cv=StratifiedKFold()



In [ ]:
from sklearn.model_selection import GridSearchCV
grid=GridSearchCV(
    estimator=model,
    cv=cv,
    param_grid=param,
    scoring="accuracy",
    n_jobs=-1
)

grid.fit(x_train_scaled,y_train)
y_pred_grid=grid.predict(x_test_scaled)
cm=confusion_matrix(y_test,y_pred_grid)
score=accuracy_score(y_test,y_pred_grid)
print(cm)
print(score)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
1800 fits failed out of a total of 2700.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
900 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py", line 54, in _check_solver
    raise ValueError(
ValueError: 

[[5610    0]
 [  10    1]]
0.9982209571250668


In [ ]:
from sklearn.model_selection import RandomizedSearchCV
rcv=RandomizedSearchCV(
    estimator=model,
    param_distributions=param,
    scoring="accuracy",
    cv=5
)
rcv.fit(x_train_scaled,y_train)
y_pred_rcv=rcv.predict(x_test_scaled)
cm=confusion_matrix(y_test,y_pred_grid)
score=accuracy_score(y_test,y_pred_grid)
print(cm)
print(score)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
40 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
25 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py", line 54, in _check_solver
    raise ValueError(
ValueError: Solve

[[5610    0]
 [  10    1]]
0.9982209571250668


In [ ]:
#accuracy from RandomisedSearchCV and GridSearchCV are same